<a href="https://www.kaggle.com/code/mikedelong/world-population-2023-maps?scriptVersionId=139223101" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
df = pd.read_csv('/kaggle/input/world-population-by-country-2023/WorldPopulation2023.csv')
df.columns=['Rank', 'Country', 'Population 2023', 'Yearly Change (%)', 'Net Change',
       'Density', 'Land Area', 'Net Migrants', 'Fertility Rate',
       'Median Age', 'Urban Population (%)', 'World Share (%)']

df.head()

,Rank,Country,Population 2023,Yearly Change (%),Net Change,Density,Land Area,Net Migrants,Fertility Rate,Median Age,Urban Population (%),World Share (%)
0,36,Afghanistan,42239854,2.70 %,1111083,65,652860,-65846,4.4,17.0,26 %,0.53 %
1,138,Albania,2832439,-0.35 %,-9882,103,27400,-8000,1.4,38.0,67 %,0.04 %
2,34,Algeria,45606480,1.57 %,703255,19,2381740,-9999,2.8,28.0,75 %,0.57 %
3,212,American Samoa,43914,-0.81 %,-359,220,200,-790,2.2,29.0,N.A.,0.00 %
4,202,Andorra,80088,0.33 %,264,170,470,200,1.1,43.0,85 %,0.00 %


In [2]:
df.tail()

,Rank,Country,Population 2023,Yearly Change (%),Net Change,Density,Land Area,Net Migrants,Fertility Rate,Median Age,Urban Population (%),World Share (%)
229,225,Wallis & Futuna,11502,-0.60 %,-70,82,140,-119,1.9,37.0,0 %,0.00 %
230,172,Western Sahara,587259,1.96 %,11273,2,266000,5600,2.2,32.0,95 %,0.01 %
231,44,Yemen,34449825,2.24 %,753211,65,527970,-29914,3.6,19.0,37 %,0.43 %
232,63,Zambia,20569737,2.76 %,552062,28,743390,-5000,4.2,17.0,46 %,0.26 %
233,74,Zimbabwe,16665409,2.11 %,344872,43,386850,-9999,3.4,18.0,37 %,0.21 %


In [3]:
df.shape

(234, 12)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234 entries, 0 to 233
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Rank                  234 non-null    int64  
 1   Country               234 non-null    object 
 2   Population 2023       234 non-null    int64  
 3   Yearly Change (%)     234 non-null    object 
 4   Net Change            234 non-null    int64  
 5   Density               234 non-null    int64  
 6   Land Area             234 non-null    int64  
 7   Net Migrants          234 non-null    int64  
 8   Fertility Rate        233 non-null    float64
 9   Median Age            233 non-null    float64
 10  Urban Population (%)  234 non-null    object 
 11  World Share (%)       234 non-null    object 
dtypes: float64(2), int64(6), object(4)
memory usage: 22.1+ KB


In [5]:
df.describe()

,Rank,Population 2023,Net Change,Density,Land Area,Net Migrants,Fertility Rate,Median Age
count,234.000000,2.340000e+02,2.340000e+02,234.000000,2.340000e+02,2.340000e+02,233.000000,233.000000
mean,117.500000,3.437565e+07,3.000230e+05,477.414530,5.559568e+05,1.301282e+01,2.414163,31.309013
std,67.694165,1.373861e+08,1.001815e+06,2320.693692,1.691024e+06,1.698334e+05,1.155913,9.628386
min,1.000000,5.180000e+02,-2.957105e+06,0.000000,0.000000e+00,-9.104750e+05,0.800000,15.000000
25%,59.250000,4.696482e+05,2.360000e+02,38.250000,2.650000e+03,-9.776750e+03,1.600000,22.000000
50%,117.500000,5.643895e+06,2.860150e+04,96.500000,7.972000e+04,-5.000000e+02,2.000000,32.000000
75%,175.750000,2.324537e+07,2.236855e+05,242.000000,4.070800e+05,4.750000e+02,3.000000,40.000000
max,234.000000,1.428628e+09,1.145449e+07,24360.000000,1.637687e+07,1.784718e+06,6.700000,54.000000


Let's make some maps; first we need to get data that will let plotly place things on a map.

In [6]:
iso_df = pd.read_csv(filepath_or_buffer='/kaggle/input/country-mapping-iso-continent-region/continents2.csv').drop(columns=['alpha-2', 'intermediate-region', 'intermediate-region-code'])
iso_df.head()

,name,alpha-3,country-code,iso_3166-2,region,sub-region,region-code,sub-region-code
0,Afghanistan,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0
1,Åland Islands,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,150.0,154.0
2,Albania,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,150.0,39.0
3,Algeria,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,2.0,15.0
4,American Samoa,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,9.0,61.0


In [7]:
from plotly.express import choropleth
def make_map(column: str):
    choropleth(data_frame=df[['Country', column]].merge(right=iso_df[['name', 'alpha-3',]], left_on='Country', right_on='name').drop(columns=['name']), color=column, locations='alpha-3', ).show()
print('defined function make_map')

defined function make_map


In [8]:
columns = [key for key, value in df.dtypes.to_dict().items() if value in ['float64', 'int64']] 
columns

['Rank',
 'Population 2023',
 'Net Change',
 'Density',
 'Land Area',
 'Net Migrants',
 'Fertility Rate',
 'Median Age']

In [9]:
make_map(column=columns[0])

In [10]:
make_map(column=columns[1])

In [11]:
make_map(column=columns[2])

In [12]:
make_map(column=columns[3])

In [13]:
make_map(column=columns[4])

In [14]:
make_map(column=columns[5])

In [15]:
make_map(column=columns[6])

In [16]:
make_map(column=columns[7])